# **Airfare Price Prediction**

Dennis Myasnyankin, Vannessa Salazar, and Christine Vu

Shiley-Marcos School of Engineering, University of San Diego

ADS 599: Capstone Project

Professor Ebrahim Tarshizi

December 11, 2023

***

## **Data Importing**

### **Import Libraries**

In [1]:
import pandas as pd
from pandas import json_normalize
from datetime import datetime
import zipfile
import io
import os
import json
import pandas as pd
from datetime import datetime
from google.colab import files
import requests
from io import BytesIO

### **Data Ingestion**

In [7]:
github_repo_url = 'https://github.com/VSbr22/Capstone/raw/main/data/jsons.zip'

In [8]:
# Fetch the zip file from the GitHub repository
response = requests.get(github_repo_url)
zip_file = BytesIO(response.content)

In [9]:
#Unzip the folder
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall('your_unzipped_folder')

In [10]:



# List all JSON files in the 'jsons' subfolder
json_subfolder = os.path.join('your_unzipped_folder', 'jsons')
json_files = [file for file in os.listdir(json_subfolder) if file.endswith('.json')]

# Initialize an empty list for extracted data
out = []

# Iterate over each JSON file
for file_name in json_files:
    # Load JSON data
    with open(os.path.join(json_subfolder, file_name), 'r') as file:
        data = json.load(file)

    # Christines code for processing
    schedule_descs = data["groupedItineraryResponse"]["scheduleDescs"]
    leg_descs = data["groupedItineraryResponse"]["legDescs"]
    itinerary_groups = data["groupedItineraryResponse"]["itineraryGroups"]

    # Iterate over itinerary groups
    for group in itinerary_groups:
        itineraries = group.get("itineraries", [])
        departure_date = group["groupDescription"]["legDescriptions"][0]["departureDate"]
        departure_city = group["groupDescription"]["legDescriptions"][0]["departureLocation"]
        arrival_date = group["groupDescription"]["legDescriptions"][0].get("arrivalDate", departure_date)
        arrival_location = group["groupDescription"]["legDescriptions"][0]["arrivalLocation"]

        # Calculate "Days Until Departure" based on current date and departure date
        current_date = datetime.now()
        departure_date = datetime.strptime(departure_date, "%Y-%m-%d")
        days_until_departure = (departure_date - current_date).days

        # Iterate over itineraries in the group
        for itinerary in itineraries:
            legs = itinerary.get("legs", [])
            for leg in legs:
                schedules = leg_descs[leg.get('ref') - 1].get("schedules", [])
                for schedule in schedules:
                    schedule_data = schedule_descs[schedule.get('ref') - 1]

                    departure_time = schedule_data["departure"]["time"].split("+")[0]
                    arrival_time = schedule_data["arrival"]["time"].split("+")[0]
                    flight_duration = schedule_data["elapsedTime"]
                    operating_carrier = schedule_data["carrier"]["operating"]
                    aircraft_type = schedule_data["carrier"]["equipment"]["code"]
                    cabin_class = itinerary["pricingInformation"][0]["fare"]["passengerInfoList"][0]["passengerInfo"]["fareComponents"][0]["segments"][0]["segment"]["cabinCode"]
                    fare_class = itinerary["pricingInformation"][0]["fare"]["passengerInfoList"][0]["passengerInfo"]["fareComponents"][0]["segments"][0]["segment"]["bookingCode"]
                    price = itinerary["pricingInformation"][0]["fare"]["totalFare"]["totalPrice"]

                    # Add to output
                    out.append([
                        departure_city, departure_date, departure_time,
                        arrival_location, arrival_date, arrival_time,
                        flight_duration, operating_carrier, aircraft_type,
                        cabin_class, fare_class, price, days_until_departure
                    ])

# Creating a DataFrame
columns = [
    "Departure City", "Departure Date", "Departure Time",
    "Arrival Location", "Arrival Date", "Arrival Time",
    "Flight Duration", "Operating Carrier", "Aircraft Type",
    "Cabin Class", "Fare Class", "Price", "Days Until Departure"
]

df = pd.DataFrame(out, columns=columns)


In [11]:
df

,Departure City,Departure Date,Departure Time,Arrival Location,Arrival Date,Arrival Time,Flight Duration,Operating Carrier,Aircraft Type,Cabin Class,Fare Class,Price,Days Until Departure
0,NRT,2023-12-15,06:05:00,FUK,2023-12-15,08:15:00,130,GK,320,Y,C,92.4,23
1,NRT,2023-12-15,07:20:00,FUK,2023-12-15,09:35:00,135,GK,320,Y,H,112.2,23
2,NRT,2023-12-15,15:00:00,FUK,2023-12-15,17:30:00,150,GK,32Q,Y,H,112.2,23
3,NRT,2023-12-15,08:20:00,FUK,2023-12-15,10:40:00,140,GK,320,Y,K,130.9,23
4,NRT,2023-12-15,10:05:00,FUK,2023-12-15,12:25:00,140,GK,320,Y,K,130.9,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8190,FCO,2023-11-22,09:30:00,PMO,2023-11-22,10:35:00,65,W2,73H,Y,W,132.0,0
8191,FCO,2023-11-22,13:35:00,PMO,2023-11-22,14:45:00,70,W2,73H,Y,W,132.0,0
8192,FCO,2023-11-22,21:45:00,PMO,2023-11-22,22:55:00,70,AZ,32N,Y,Q,146.4,0
8193,FCO,2023-11-22,17:25:00,PMO,2023-11-22,18:30:00,65,W2,73H,Y,U,153.0,0
